In [1]:
import json
import os
from utils import nlp
import pandas as pd
import random
from random import shuffle, sample
random.seed(30)
from collections import Counter
import numpy as np


In [ ]:
#Auxiliar functions
def slice_keys(key_list, proportion=0.3):
    return list(key_list)[:int(len(key_list) * proportion)]

def scramble(sentence):
    split = sentence.split()  # Split the string into a list of words
    shuffle(split)  # This shuffles the list in-place.
    return ' '.join(split)  # Turn the list back into a string

# Generate Dataset for Language Modeling GPT
Using the Multiwoz dataset to fine tuning gpt model to generate syntetic utterances

In [2]:
mw_base_dir = '../MultiWOZ_2.1/'

In [3]:
mw_base_dir = '../MultiWOZ_2.1/'
with open(mw_base_dir + 'valListFile.txt', 'r') as fp:
    val_list = fp.readlines()
    val_list = [elem.strip() for elem in val_list]

In [4]:
with open(mw_base_dir + 'testListFile.txt', 'r') as fp:
    test_list = fp.readlines()
    test_list = [elem.strip() for elem in test_list]

In [5]:
mw_data = json.load(open(mw_base_dir + 'data.json', 'r'))

In [6]:
train_list = set(mw_data.keys()) - set(test_list) - set(val_list)

In [ ]:
def generate_dataset_lm_rnn(keys, filename):
    examples = []
    for key in keys:
        data = mw_data[key]['log']
        for i in range(0, len(data), 2):
            sys = data[i + 1]['text']
            examples.append(nlp.normalize(sys))
    with open(filename+'.txt', 'w') as f:
        for item in examples:
            f.write("%s \n" % item)
    
def generate_dataset_lm(keys, filename, add_label=False):
    examples = []
    for key in keys:
        data = mw_data[key]['log']
        for i in range(0, len(data), 2):
            sys = data[i + 1]['text']
            if add_label:
                examples.append((nlp.normalize(sys), '1'))
            else:
                examples.append(nlp.normalize(sys))
    with open(filename+'_lm.raw', 'w') as f:
        for item in examples:
            f.write("%s <|endoftext|> \n" % item)

In [ ]:
generate_dataset_lm(train_list, 'train')
#generate_dataset_lm(val_list, 'valid')
#generate_dataset_lm(test_list, 'test')

In [ ]:
len(test_list)

# Pre process the utterances generated by SC-GPT
Extract the results from sc-gpt to calculate the coherence of the model

In [2]:
pred = json.load(open('multiwoz.pred.txt'))

In [7]:
test = []
for key in train_list:
    data = mw_data[key]['log']
    for i in range(0, len(data), 2):
        sys = data[i + 1]['text']
        test.append((nlp.normalize(sys).replace('"', ''), '1'))

In [8]:
len(test)

56778

In [4]:
gen_utt = []
for examples in pred:
    utt = examples[0]
    cl_idx = utt.find('<|endoftext|>')
    utt = utt[:cl_idx].strip().lower()
    gen_utt.append((nlp.normalize(utt), '0'))


In [5]:
len(gen_utt)

7372

In [6]:
df = pd.DataFrame(gen_utt)
df.columns = ['text', 'label']
df = df.sample(frac=1)

,text,label
4326,i have booked it for you . your reference numb...,0
2889,"thank you , goodbye !",0
4668,what day and time would you like to book ?,0
133,you are welcome ! have a great day !,0
254,thank you for using our services . have a grea...,0
...,...,...
2013,thank you for using the cambridge restaurant p...,0
1993,thank you for contacting us . have a great day .,0
2036,the first train leaves at 5:24 .,0
521,thank you for contacting cambridge towninfo ce...,0


In [7]:
df.to_csv('sc_gpt_generated_test.csv', index=False)

# Generate examples for Binary classification GPT

In [ ]:
gpt_gen_train = json.load(open('lm_gpt/gpt_generated_train.json'))
gpt_gen_test = json.load(open('lm_gpt/gpt_generated_test.json'))
gpt_gen_valid = json.load(open('lm_gpt/gpt_generated_valid.json'))

In [ ]:
gpt_gen_train

In [ ]:
#BUILD DATASET FOR COHERENCE CLASSIFICATION
train = []
for key in val_list:
    data = mw_data[key]['log']
    for i in range(0, len(data), 2):
        sys = data[i + 1]['text']
        train.append((nlp.normalize(sys).replace('"', ''), '1'))
        #train.append((scramble(nlp.normalize(sys)), '0'))

In [ ]:
len(train)

In [ ]:
lite = []
for elem in gpt_gen_valid:
    idx = elem.find('<|endoftext|>')
    if idx != -1:
        elem = elem[:idx]
    lite.append((nlp.normalize(elem).replace('"', ''), '0'))

In [ ]:
train += lite + random.sample(lite, 221)

In [ ]:
import pandas as pd
df = pd.DataFrame(train, columns=['text','label'])
df.to_csv('valid_bert_gpt.csv', index=False)

# Generate examples for Binary classification Transformer based LM

In [18]:
base_trans = 'lm_transformer/'
fin = open(base_trans + 'valid_transformer.txt', 'r')
data = fin.read()
trans_data = data.split('<eos>')
trans_data = [elem.replace('\n', ' ').strip() for elem in trans_data]

In [19]:
len(trans_data)

8781

In [20]:
trans_data = [w for w in trans_data if len(w.split()) > 2]
trans_data = trans_data[:7374]

In [21]:
train = []
for key in val_list:
    data = mw_data[key]['log']
    for i in range(0, len(data), 2):
        sys = data[i + 1]['text']
        train.append((nlp.normalize(sys).replace('"', ''), '1'))

In [22]:
len(trans_data)

7374

In [23]:
for elem in trans_data:
    train.append((nlp.normalize(elem), '0'))

In [24]:
len(train)

14748

In [25]:
df = pd.DataFrame(train, columns=['text','label'])
df = df.sample(frac=1)
df.to_csv('valid_bert_transformer.csv', index=False)

# Generate examples for Binary classification LSTM

In [ ]:
base_lstm = 'lm_lstm/'
fin = open(base_lstm + 'generated_lstm_valid.txt', 'r')
data = fin.read()
lstm_data = data.split('<eos>')
lstm_data = [elem.replace('\n', ' ').strip() for elem in lstm_data]

In [ ]:
len(lstm_data)

In [ ]:
lstm_data = lstm_data[:7374]

In [ ]:
train = []
for key in val_list:
    data = mw_data[key]['log']
    for i in range(0, len(data), 2):
        sys = data[i + 1]['text']
        train.append((nlp.normalize(sys).replace('"', ''), '1'))
        #train.append((scramble(nlp.normalize(sys)), '0'))

In [ ]:
len(train)

In [ ]:
for elem in lstm_data:
    train.append((nlp.normalize(elem), '0'))

In [ ]:
df = pd.DataFrame(train, columns=['text','label'])
df = df.sample(frac=1)
df.to_csv('valid_bert_lstm.csv', index=False)

In [ ]:
df.shape

# Generate examples mixing GPT and LSTM

In [ ]:
gpt_gen_train = json.load(open('lm_gpt/gpt_generated_train.json'))
gpt_gen_test = json.load(open('lm_gpt/gpt_generated_test.json'))
gpt_gen_valid = json.load(open('lm_gpt/gpt_generated_valid.json'))

In [ ]:
lite = []
for elem in gpt_gen_valid:
    idx = elem.find('<|endoftext|>')
    if idx != -1:
        elem = elem[:idx]
    lite.append((nlp.normalize(elem).replace('"', ''), '0'))

In [ ]:
len(lite)

In [ ]:
base_lstm = 'lm_lstm/'
fin = open(base_lstm + 'generated_valid.txt', 'r')
data = fin.read()
lstm_data = data.split('<eos>')
lstm_data = [elem.replace('\n', ' ').strip() for elem in lstm_data]

In [ ]:
train_lstm = []
for elem in lstm_data:
    train_lstm.append((nlp.normalize(elem), '0'))

In [ ]:
len(train_lstm)

In [ ]:
len(train)//2

In [ ]:
full = train + train_lstm[:3687] + lite[:3687]

In [ ]:
shuffle(full)

In [ ]:
df = pd.DataFrame(full, columns=['text','label'])
df = df.sample(frac=1)
df.to_csv('valid_bert_lstm_gpt.csv', index=False)

# Generate examples mixing LSTM GPT TRANSFORMER

In [74]:
# GPT LM
gpt_gen_train = json.load(open('lm_gpt/gpt_generated_train.json'))
gpt_gen_test = json.load(open('lm_gpt/gpt_generated_test.json'))
gpt_gen_valid = json.load(open('lm_gpt/gpt_generated_valid.json'))

In [75]:
# LSTM language model
base_lstm = 'lm_lstm/'
fin = open(base_lstm + 'generated_lstm_valid.txt', 'r')
data = fin.read()
lstm_data = data.split('<eos>')
lstm_data = [elem.replace('\n', ' ').strip() for elem in lstm_data]

In [76]:
# Transformers LM
base_trans = 'lm_transformer/'
fin = open(base_trans + 'valid_generated_transformer.txt', 'r')
data = fin.read()
trans_data = data.split('<eos>')
trans_data = [elem.replace('\n', ' ').strip() for elem in trans_data]

In [77]:
train_2 = random.sample(trans_data, 2458) + random.sample(gpt_gen_train, 2458) + random.sample(trans_data, 2458)

In [78]:
# MULTIWOZ
train = []
for key in val_list:
    data = mw_data[key]['log']
    for i in range(0, len(data), 2):
        sys = data[i + 1]['text']
        train.append((nlp.normalize(sys).replace('"', ''), '1'))


In [73]:
len(train)

7374

In [67]:
len(train_2)

7372

In [79]:
for elem in train_2:
    train.append((nlp.normalize(elem), '0'))

In [81]:
len(train_2)

7374

In [82]:
shuffle(train)

In [83]:
df = pd.DataFrame(train, columns=['text','label'])
df = df.sample(frac=1)
df.to_csv('valid_bert_mixed.csv', index=False)

# Most frequent prompts system

In [ ]:
train = []
for key in train_list:
    data = mw_data[key]['log']
    for i in range(0, len(data), 2):
        sys = data[i + 1]['text']
        train.append((nlp.normalize(sys), '1'))

In [ ]:
len(train)

In [ ]:
df = pd.DataFrame(train)
df = df.rename(columns={0:'text', 1:'label'})
df['prompts'] = df.text.map(lambda x: ' '.join(x.split()[:2]))
df = df.groupby('prompts').count().sort_values('label', ascending=False).reset_index()
df.drop(df[['text','label']], axis=1, inplace=True)
newdf = pd.DataFrame(np.repeat(df.values,14,axis=0))
newdf.columns = df.columns
newdf = newdf.sample(frac=1)
newdf.to_csv('prompts_train.csv', index=False)

In [ ]:
newdf.shape

# HDSA Lexicalization to calculate the coherence

In [10]:
os.listdir('hdsa')

['test.json', 'test_hdsa.csv', 'results.txt.pred.BERT_dim128_w_domain.pred']

In [11]:
hdsa_test = json.load(open('hdsa/test.json'))

In [12]:
dacts = {}
for elem in hdsa_test:
    diag_list = [] 
    for turn in elem['info']:
        diag_list.append(turn['act'])
    dacts[elem['file']] = diag_list

In [13]:
generated_hdsa = json.load(open('hdsa/results.txt.pred.BERT_dim128_w_domain.pred'))

In [15]:
#replacements like hotels => hotel -s
fin = open('hdsa/mapping.pair')
replacements = []
for line in fin.readlines():
    tok_from, tok_to = line.replace('\n', '').split('\t')
    replacements.append((' ' + tok_from + ' ', ' ' + tok_to + ' '))

In [17]:
dialogue_acts = {}
value_count = '[value_count]'
for fname, lst_dact in dacts.items():
    list_dact = []
    for dact in lst_dact:
        formatted_da = {}
        for slot, value in dact.items():
            if slot.split('-')[-1] == 'choice':
                formatted_da[value_count] = value
            elif value != 'none' and value != '?':
                key = '['+ slot.split('-')[0] + '_' + slot.split('-')[-1] + ']'
                for fromx, tox in replacements:
                    value = ' ' + value + ' '
                    value = value.replace(fromx, tox)[1:-1]
                formatted_da[key] = value
        list_dact.append(formatted_da)
    dialogue_acts[fname] = list_dact


In [18]:
# Replace the formated dialogue acts slot value into the utterances
cleaned = []
for fname, utts in generated_hdsa.items():
    for utterance, das in zip(utts, dialogue_acts[fname]):
        for slot, value in das.items():
            utterance = utterance.replace(slot, value)
        cleaned.append((utterance, '0'))

In [19]:
cleaned

[('there are 23 hotel -s that meet your criteria . do you have a preference for area ?',
  '0'),
 ('there are 9 guesthouses in the various . the [hotel_name] is [hotel_pricerange] -ly priced .',
  '0'),
 ('i would recommend the [hotel_name] . would you like me to book it for you ?',
  '0'),
 ('i can help you with that . what is your stay ?', '0'),
 ('i can book it for you .', '0'),
 ('yes , it is . would you like me to book it for you ?', '0'),
 ('your booking was successful . the table will be reserved for [value_count] minutes . your reference number is [restaurant_reference] . is there anything else i can help you with ?',
  '0'),
 ('you are welcome . have a great day !', '0'),
 ('nandos is a [restaurant_pricerange] [restaurant_food] restaurant in the city centre of town at 33-34 saint andrews street . would you like me to book a table for you ?',
  '0'),
 ('your booking was successful . the table will be reserved for [value_count] minutes . your reference number is [restaurant_refe

In [20]:
hdsa_df = pd.DataFrame(cleaned, columns=['text','label'])

hdsa_df.to_csv('test_hdsa.csv', index=False)